In [33]:
!rm -r /kaggle/working/NMT-MultiModel-Framework
!git clone https://github.com/abdo-ashraf/NMT-MultiModel-Framework.git

Cloning into 'NMT-MultiModel-Framework'...
remote: Enumerating objects: 517, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 517 (delta 69), reused 88 (delta 35), pack-reused 390 (from 1)
Receiving objects: 100% (517/517), 106.34 KiB | 6.65 MiB/s, done.
Resolving deltas: 100% (308/308), done.


In [2]:
import json
import os
!pip -q install -r /kaggle/working/NMT-MultiModel-Framework/requirements.txt

In [31]:
for_now_datas = ["ar-en", "sp-en", "de-en"]
needed_data = for_now_datas[1]

## Datas

In [15]:
out_dir = './out/data/'
data_type = 'opus'
maxlen = 25
seed = 123
valid_test_split = 0.1

if needed_data == "ar-en":
    ## ar-en
    !python /kaggle/working/NMT-MultiModel-Framework/make_data/ar_en_data_workflow.py \
    --out_dir $out_dir \
    --data $data_type \
    --maxlen $maxlen \
    --seed $seed \
    --valid_test_split $valid_test_split

elif needed_data == "sp-en":
    ## sp-en
    !python /kaggle/working/NMT-MultiModel-Framework/make_data/sp_en_data_workflow.py \
    --out_dir $out_dir \
    --seed $seed \
    --valid_test_split $valid_test_split
    
elif needed_data == "de-en":
    ## de-en
    import pandas as pd
    import os
    
    # with open('/kaggle/input/machine-translation-with-multi30k-de-en/.data/Multi30k/train.en') as f:
    #     en_train = f.read().strip().split('\n')
    #     en_train = [train.strip().lower() for train in en_train]
    
    # with open('/kaggle/input/machine-translation-with-multi30k-de-en/.data/Multi30k/train.de') as f:
    #     de_train = f.read().strip().split('\n')
    #     de_train = [train.strip().lower() for train in de_train]
    
    
    # with open('/kaggle/input/machine-translation-with-multi30k-de-en/.data/Multi30k/val.en') as f:
    #     en_valid = f.read().strip().split('\n')
    #     en_valid = [valid.strip().lower() for valid in en_valid]
    
    # with open('/kaggle/input/machine-translation-with-multi30k-de-en/.data/Multi30k/val.de') as f:
    #     de_valid = f.read().strip().split('\n')
    #     de_valid = [valid.strip().lower() for valid in de_valid]
    
    
    # dict_train = {'source_lang': en_train,
    #             'target_lang': de_train}
    
    # dict_valid = {'source_lang': en_valid,
    #             'target_lang': de_valid}
    
    # os.makedirs(out_dir, exist_ok=True)
    
    # df_train = pd.DataFrame(dict_train).dropna(axis=1)
    # df_valid = pd.DataFrame(dict_valid).dropna(axis=1)
    
    # df_valid.to_csv(os.path.join(data_out_dir,'de-en_valid.csv'), index=False)
    # df_train.to_csv(os.path.join(data_out_dir,'de-en_train.csv'), index=False)

(107067, 2), (5949, 2), (5948, 2)


In [16]:
import os
train_csv_path = os.path.join(out_dir, f'{needed_data}_train.csv')
valid_csv_path = os.path.join(out_dir, f'{needed_data}_valid.csv')
test_csv_path = os.path.join(out_dir, f'{needed_data}_test.csv')

## Tokenizers

In [34]:
tokenizer_path = f'./out/tokenizers/{needed_data}_tokenizer'

# Define the JSON content
tokenizer_config = {
    "model_prefix": tokenizer_path,
    "model_type": "bpe",
    "vocab_size": 6000,
    "normalization_rule_name": "nmt_nfkc_cf",
    "remove_extra_whitespaces": True,
    "input_sentence_size": 200000000,
    "max_sentence_length": 4192,
    "seed_sentencepiece_size": 1000000,
    "shuffle_input_sentence": True,
    "character_coverage": 0.99995,
    "byte_fallback": True,
    "split_digits": True,
    "split_by_unicode_script": True,
    "split_by_whitespace": True,
    "split_by_number": True,
    "max_sentencepiece_length": 16,
    "add_dummy_prefix": True,
    "allow_whitespace_only_pieces": True,
    "unk_id": 1,
    "bos_id": 2,
    "eos_id": 3,
    "pad_id": 0,
    "num_threads": os.cpu_count(),
}

# Write the content to a JSON file

tokenizers_config_path = f'./{needed_data}_tokenizers_config.json'
with open(tokenizers_config_path, 'w') as json_file:
    json.dump(tokenizer_config, json_file, indent=4)

In [35]:
col1, col2 = needed_data.split('-')

!python /kaggle/working/NMT-MultiModel-Framework/Tokenizers/tokenizers_workflow.py \
--train_csv_path $train_csv_path \
--train_on_columns $col1 $col2 \
--config_path $tokenizers_config_path


Starting Tokenizer Train...
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input_format: 
  model_prefix: ./out/tokenizers/sp-en_tokenizer
  model_type: BPE
  vocab_size: 6000
  self_test_sample_size: 0
  character_coverage: 0.99995
  input_sentence_size: 200000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 4
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 1
  bos_id: 2
  eos_id: 3
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enabl

## Model Training

In [ ]:
model_types = ['s2s', 's2sattention', 'transformer']
model_type = model_types[2]

In [12]:
model_config = {
    "model_type": model_type,
    "dim_embed": 256,
    "dim_model": 256,
    "dim_feedforward": 1024,
    "num_layers": 4,
    "dropout": 0.1,
    "maxlen": 512,
    "flash_attention": False
}
# Write the content to a JSON file
model_config_path = f'./{model_type}_model_config.json'
with open(model_config_path, 'w') as json_file:
    json.dump(model_config, json_file, indent=4)

In [32]:
max_steps = 6000
training_config = {
    "save_models_dir": "./out/models",
    "save_plots_dir": "./out/plots",
    "learning_rate": 1e-3,
    "max_steps": max_steps,
    "seed": 123,
    "precision": "high",
    "device": "cuda",
    "batch_size": 128,
    "cpu_num_workers": os.cpu_count(),
    "weight_decay": 0.00001,
    "onnx": False,
    "run_name": model_type,
    "pin_memory": True,
    "warmup_steps": 1000,
    "save_steps": 2000,
    "torch_compile": False,
    "eval_steps": 1000,
    "lr_decay_ratio": 0.01
}
# Write the content to a JSON file
training_config_path = f'./{model_type}_training_config.json'
with open(training_config_path, 'w') as json_file:
    json.dump(training_config, json_file, indent=4)

In [14]:
tokenizer_model = tokenizer_path + '.model'

!python /kaggle/working/NMT-MultiModel-Framework/workflow.py \
--train_csv_path $train_csv_path \
--valid_csv_path $valid_csv_path \
--source_column_name 'en' \
--target_column_name 'sp' \
--tokenizer_path $tokenizer_model \
--model_config_path $model_config_path \
--training_config_path $training_config_path

---------------------Starting Tokenizer Loading...---------------------
Tokenizer length 6000
Tokenizer Loading Done.
---------------------Starting Data Loading...---------------------
Training data length 83,276, Validation data length 17,844
Source tokens: tensor([ 564,  895,  396,  301,  739, 2630, 1602,  832, 1220, 5979])
Target tokens: tensor([   2, 1496,  692,  485, 1212, 3573,  268,  769,  832, 5951,    3])
Data Loading Done.
---------------------Parsing Model arguments...---------------------
ModelArgs(
model_type=transformer,
dim_embed=256,
dim_model=256,
dim_feedforward=1024,
num_layers=4,
dropout=0.1,
maxlen=512,
flash_attention=False
)
Parsing Done.
---------------------Loading the model...---------------------
Module                         Trainable  Non-Trainable
embed_shared_src_trg_cls       1,536,000              0
positonal_shared_src_trg         131,072              0
dropout                                0              0
transformer_encoder            3,159,040   

## Testing

In [15]:
import sys
sys.path.append('/kaggle/working/NMT-MultiModel-Framework')

In [16]:
import torch
state_dict = torch.load(f'/kaggle/working/out/models/{model_type}_step_{max_steps}.pth', weights_only=True)
model_state_dict = state_dict['model_state_dict']
optimizer_state_dict = state_dict['optimizer_state_dict']

In [17]:
from Models.AutoModel import get_model
from Models.ModelArgs import ModelArgs
from Tokenizers.Tokenizers import Callable_tokenizer

tokenizer = Callable_tokenizer(tokenizer_path+'.model')

model_args = ModelArgs(model_config_path)
model = get_model(model_args, len(tokenizer))
model.load_state_dict(model_state_dict)
model.eval()
print('Done')

Done


In [21]:
def en_to_sp(text:str):
    in_tokens = tokenizer(text)
    out_tokens = model.translate(torch.tensor(in_tokens),
                                 tokenizer.get_tokenId('<s>'),
                                 tokenizer.get_tokenId('</s>'),
                                 pad_tokenId=tokenizer.get_tokenId('<pad>'),
                                 max_tries=30)
    return out_tokens

In [27]:
target = "¿Tom me está mirando?".lower()
text = "Is Tom looking at me?".lower()
tokens = en_to_sp(text)
print(f"English: {text}")
print(f"Spanish: {target}")
print(f"Model  : {tokenizer.decode(tokens)}")
print('-'*20)

target = "Mali es uno de los países más pobres de África subsahariana.".lower()
text = "Mali is one of the poorest countries in Subsaharan Africa.".lower()
tokens = en_to_sp(text)
print(f"English: {text}")
print(f"Spanish: {target}")
print(f"Model  : {tokenizer.decode(tokens)}")
print('-'*20)

target = "¿Cuándo fue la última vez que nos vimos?".lower()
text = "When was the last time we met?".lower()
tokens = en_to_sp(text)
print(f"English: {text}")
print(f"Spanish: {target}")
print(f"Model  : {tokenizer.decode(tokens)}")
print('-'*20)

English: is tom looking at me?
Spanish: ¿tom me está mirando?
Model  : tom me está mirando
--------------------
English: mali is one of the poorest countries in subsaharan africa.
Spanish: mali es uno de los países más pobres de áfrica subsahariana.
Model  : el mal ánimo es uno de los países más en área
--------------------
English: when was the last time we met?
Spanish: ¿cuándo fue la última vez que nos vimos?
Model  : cuándo fue la última vez que nos encontramos
--------------------


In [ ]:
# import unicodedata
# import re
# def unicodeToAscii(s):
#     return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

# def clean_ar(text):
#     text = unicodeToAscii(text) # Remove diacritics "التشكيل"
#     text = text.replace(",", "،")
#     text = text.replace(".", "۔")
#     text = text.replace("?", "؟")
#     text = re.sub(r"[;:]", "؛", text)
#     text = re.sub(r"[^؀-ۿ0-9.!؟،؛:\-()\"' ]+", " ", text)
#     text = re.sub(r'\s+', ' ', text).strip() # Trim multiple whitespaces to one
#     return text